##### Support Vector Regressor Implementation

For SVR problem statement we will be using `tips` dataset. It is an built-in dataset that comes in seaborn.

**Problem Statement**:

*We have to predict the total bill from all the given independent features*

* **Independent Features**: 
* tip: The tip amount for particular person.
* sex: Gender (Male/Female)
* day: Weekday (Sunday, Monday, Tuesday, Wednesday, Friday, Saturday, Sunday)
* smoker: Does the individual smoke (Yes/No)
* time: (Dinner/Lunch)

* **Dependent Features**:
* total_bill: The total amount we have to predict

---

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [43]:
df = sns.load_dataset('tips')

In [44]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


* `total_bill`, `tip` are of integer type
* `sex` `smoker`, `day`, time` are of object type
* `size` is of interger type

* Dataset has 244 rows and any none of those rows have null values

---

In [46]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

* There are 157 Male's and 87 Female's

---

In [47]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

* There are 151 people who don't smoke and 93 that smoke.

---

In [48]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

* We have only four days information : `Sat`, `Sun`, `Thu`, `Fri`

---

In [49]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

Since we have categorical features `day`, `time`, `sex` and `smoker` we need to perform Feature Encoding:

* For `time`, `sex`, and `smoker` we can use **Label Encoding**
* For `day` (since it has more than 2 features) we can use **OneHotEncoding**

In [50]:
# Divide the data as independent and dependent features
X=df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y=df['total_bill']

In [51]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [52]:
X_train.head()

,tip,sex,smoker,day,time,size
234,3.00,Male,Yes,Sat,Dinner,2
227,3.00,Male,No,Sat,Dinner,4
180,3.68,Male,Yes,Sun,Dinner,4
5,4.71,Male,No,Sun,Dinner,4
56,3.00,Male,Yes,Sat,Dinner,4


* **Now we Start Feature Encoding:**

The reason we perform splitting of data before Feature Encoding because there is always a possibility of data leakage. By doing train-test split before encoding helps us avoid that

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [55]:
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [56]:
X_train.head()

,tip,sex,smoker,day,time,size
234,3.00,1,1,Sat,0,2
227,3.00,1,0,Sat,0,4
180,3.68,1,1,Sun,0,4
5,4.71,1,0,Sun,0,4
56,3.00,1,1,Sat,0,4


In [57]:
# same transformation needs to applied to test dataset
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [58]:
X_test.head()

,tip,sex,smoker,day,time,size
24,3.18,1,0,Sat,0,2
6,2.00,1,0,Sun,0,2
153,2.00,1,0,Sun,0,4
211,5.16,1,1,Sat,0,4
198,2.00,0,1,Thur,1,2


In [59]:
# Now OneHotEncoding for 'day' column using ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [61]:
ct = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop='first'), [3])], remainder='passthrough')

In [64]:
X_train = ct.fit_transform(X_train)

In [63]:
X_test = ct.transform(X_test)

In [65]:
# Now, we can apply SVR
from sklearn.svm import SVR
svr = SVR()

In [66]:
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)

In [67]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("R2 Score: ", r2_score(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.496241694353699
Mean Squared Error:  35.08995489977148
Mean Absolute Error:  4.175959078886017


In [68]:
# Hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C':[0.1, 1, 10, 100, 1000],
              'gamma':[1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [69]:
gridcv = GridSearchCV(estimator=SVR(), param_grid=param_grid, cv = 5, verbose=3, refit=True)

In [70]:
# fit the model 
gridcv.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.019 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.050 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.002 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.081 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.102 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.095 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.037 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.073 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.033 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.005 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

,estimator,SVR()
,param_grid,"{'C': [0.1, 1, ...], 'gamma': [1, 0.1, ...], 'kernel': ['rbf']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'rbf'


In [75]:
gridcv.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [71]:
y_pred = gridcv.predict(X_test)

In [72]:
print("R2 Score: ", r2_score(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))

R2 Score:  0.5639365600404469
Mean Squared Error:  30.374578979871718
Mean Absolute Error:  3.9374191083098378
